#İmport



In [61]:
import pandas as pd


#Dataset

In [62]:
movies=pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


#Regex

In [63]:
import re

def clean_title(title):
  title=re.sub("[^a-zA-Z0-9]"," ",title)
  return title


In [64]:
movies["clean_title"]=movies["title"].apply(clean_title)
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(ngram_range=(1,2))
tfidf=vectorizer.fit_transform(movies["clean_title"])

In [66]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
def search(title):
  title=clean_title(title)
  query_vec=vectorizer.transform([title])
  similarity=cosine_similarity(query_vec,tfidf).flatten()
  indices=np.argpartition(similarity,-5)[-5:]
  results=movies.iloc[indices].iloc[::-1]
  return results


In [67]:
from os import name
import ipywidgets as widgets
from IPython.display import display
movie_input=widgets.Text(
    value="Harry porter",
    description='Movie Title',
    disabled=False
)
movie_list = widgets.Output()
def on_type(data):
  with movie_list:
    movie_list.clear_output()
    title=data["new"]
    if len(title)>5:
      display(search(title))
movie_input.observe(on_type,names="value")

display(movie_input,movie_list)



Text(value='Harry porter', description='Movie Title')

Output()

In [68]:
ratings=pd.read_csv('ratings.csv')
movieId=1

In [69]:
similar_users=ratings[(ratings['movieId']==movieId)&(ratings["rating"]>4)]["userId"].unique()


In [70]:
similar_users_rating=ratings[(ratings["userId"].isin(similar_users))&(ratings["rating"]>4)]["movieId"]
similar_users_rating

5101         1
5105        34
5111       110
5114       150
5127       260
          ... 
127424    3671
127431    3751
127432    3760
127434    3812
127436    5060
Name: movieId, Length: 7345, dtype: int64

In [71]:

similar_users_rating=similar_users_rating.value_counts()/len(similar_users)
similar_users_rating = similar_users_rating[similar_users_rating > .10]

In [72]:
all_users=ratings[(ratings["movieId"].isin(similar_users_rating.index))&(ratings["rating"]>4)]
all_users_rec=ratings["movieId"].value_counts()/len(all_users["userId"].unique())


In [73]:
rec_percentages=pd.concat([similar_users_rating,all_users_rec],axis=1)
rec_percentages.columns=["similar","all"]
rec_percentages

,similar,all
1,1.000000,0.364497
318,0.423077,0.501775
593,0.326923,0.471006
260,0.317308,0.424852
2571,0.298077,0.450888
...,...,...
93242,NaN,0.001183
93024,NaN,0.001183
92829,NaN,0.001183
92804,NaN,0.001183


In [75]:
rec_percentages["score"]=rec_percentages["similar"]/rec_percentages["all"]
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [76]:
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")
rec_percentages

,similar,all,score
1,1.000000,0.364497,2.743506
745,0.125000,0.069822,1.790254
3114,0.230769,0.162130,1.423358
78499,0.125000,0.088757,1.408333
1028,0.125000,0.095858,1.304012
...,...,...,...
93242,NaN,0.001183,NaN
93024,NaN,0.001183,NaN
92829,NaN,0.001183,NaN
92804,NaN,0.001183,NaN


from matplotlib import pyplot as plt
rec_percentages['similar'].plot(kind='hist', bins=20, title='similar')
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.tight_layout()

from matplotlib import pyplot as plt
rec_percentages['all'].plot(kind='hist', bins=20, title='all')
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.tight_layout()

from matplotlib import pyplot as plt
rec_percentages['score'].plot(kind='hist', bins=20, title='score')
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.tight_layout()

from matplotlib import pyplot as plt
plt.figure(figsize=(6, 6))
rec_percentages.plot(kind='scatter', x='similar', y='all', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.tight_layout()

from matplotlib import pyplot as plt
plt.figure(figsize=(6, 6))
rec_percentages.plot(kind='scatter', x='all', y='score', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.tight_layout()

from matplotlib import pyplot as plt
rec_percentages['similar'].plot(kind='line', figsize=(8, 4), title='similar')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.tight_layout()

from matplotlib import pyplot as plt
rec_percentages['all'].plot(kind='line', figsize=(8, 4), title='all')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.tight_layout()

from matplotlib import pyplot as plt
rec_percentages['score'].plot(kind='line', figsize=(8, 4), title='score')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.tight_layout()

In [77]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_rating = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_rating =  similar_user_rating.value_counts() / len(similar_users)

    similar_user_rating =  similar_user_rating[ similar_user_rating> .10]
    all_users = ratings[(ratings["movieId"].isin( similar_user_rating.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([ similar_user_rating, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [78]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()